In [6]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [7]:
from BSDE import *
import warnings 
import bokeh
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_notebook, show, output_file
import time
warnings.filterwarnings("ignore")

In [11]:
T = 0.25
K = 100.
S0 = 100.
sigma = 0.2
p = 20
M = np.eye(p)
r = 0.01
mu = 0.05
R = 0.06
Q = 0.

In [18]:
N = 10000
m = 6
RF_n_trees = 200
RF_max_leaf_nodes = 1000
M_run = 5
n_picard = 3

In [ ]:
a = np.zeros(M_run)
timing = np.zeros_like(a)
for i in range (M_run): 
    start = time.time()
    test = BsdeHD(T, K, M, mu, Q, sigma, S0, r, R)
    a[i] = test.get_price(N, m, option_type='call combination', option_payoff='geometric',
                          oType='European',n_picard=n_picard, RF_n_estimators = RF_n_trees, 
                          RF_max_leaf_nodes=300)
    elapsed = time.time() - start
    timing[i] = round(elapsed, 3)

In [ ]:
means

In [ ]:
range_leaf = len(list(range(50, 10000, 50)))
a = np.zeros([range_leaf, 10])
timing = np.zeros_like(a)
for i, leaf in enumerate(list(range(50, 10000, 50))): 
    for j in range(10): 
        test = BsdeHD(T, K, M, mu, Q, sigma, S_init, r, R)
        a[i, j] = round(test.get_price(N, m, option_type='call', option_payoff='geometric',
                                       oType='European',n_picard=n_picard, RF_n_estimators = 200, RF_max_leaf_nodes=leaf), 3)      
    print (a[i])   

In [17]:
a

array([ 6.78911591,  6.78756004,  6.80368789,  6.81317811,  6.78618382])

In [ ]:
import pandas as pd

In [ ]:
param = ['q', 'mu', 'R ', 'r ','sigma', 'T', 'K', 'S0']
values = [Q, mu, R, r, sigma, T, K, S0]
dataset = list(zip(param,values))
df_param = pd.DataFrame(data = dataset, columns=['Parameters', 'values'])

In [ ]:
sparam = ['N_particles', 'm_discretization', 'N_run', 'n_picard','Number of trees', 'Max Leaf nodes']
svalues = [N, m, M_run, n_picard, RF_n_trees, RF_max_leaf_nodes]
sp_dataset = list(zip(sparam,svalues))
df_sparam = pd.DataFrame(sp_dataset, columns=['Specific Parameters', 'values'])
df_sparam

In [ ]:
run_id = ["run {}".format(i) for i in range(1, M_run+1)]
df = pd.DataFrame(list(zip(run_id, a, timing)), columns = ['run','values', 'time'])

In [ ]:
min_a = round(df['values'].min(), 4)
max_a = round(df['values'].max(), 4)
mean_a = round(df['values'].mean(), 4)
std_a = round(df['values'].std(), 4)
left = round(mean_a - 1.96 * std_a / M_run, 4)
right = round(mean_a + 1.96 * std_a / M_run, 4)

In [ ]:
stat_param = ['mean', 'std', '95% confidence interval', 'min', 'max']
stat_values = [mean_a, std_a, [left, right], min_a, max_a]
stat_data = list(zip(stat_param, stat_values))
stat_df = pd.DataFrame(stat_data, columns = ['stat parameter', 'values'])
stat_df

In [ ]:
df_join = pd.concat([df_param, df_sparam, df, stat_df], axis = 0)
df_join.to_csv('output_rf.csv')